In [1]:
import numpy as np
import string
from functools import partial
from collections import defaultdict
import pandas as pd

In [2]:
def rec_dd(depth=0):
    if depth == 2:
        return 0
    
    return defaultdict(partial(rec_dd, depth + 1))

In [3]:
initial = {} # start of a phrase (first word)
first_order = rec_dd()
second_order = rec_dd()

In [4]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [5]:
def add2dict(d, k, v):  #dictionary, key, value = list of possible next words
    if k not in d:
        d[k] = []
    d[k].append(v)   # [cat, cat, dog, dog, dog, mouse, ...]

In [6]:
mc_quotes = pd.read_csv("mckenna_quotes.csv")
for row in mc_quotes["quotes"]:    # traverse poems, populate dictionary
    tokens = remove_punctuation(row.rstrip().lower()).split()    # turns every line into list of words

    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            initial[t] = initial.get(t, 0.) + 1 # if t does not exist in dict, it gets created (assigned value 0, immediately added 1)
        
        else:
            t_1 = tokens[i-1]
            
            if i == T - 1:   # checking if end of sentence (starts counting at 0, thus T-1)
                if 'END' not in second_order[(t_1, t)]:
                    second_order[(t_1, t)]['END'] = 0 
                second_order[(t_1, t)]['END'] += 1    
                
            if i == 1:  # when given only the first word
                first_order[t_1][t] += 1
                
            else:
                t_2 = tokens[i-2]
                
                if t not in second_order[(t_1, t)]:
                    second_order[(t_2, t_1)][t] = 0
                second_order[(t_2, t_1)][t] += 1

In [10]:
# normalize the distributions (turning counts into percentage of total)
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total # maximum likelihood estimate
    

for t, c in first_order.items():    # new in ver2
    first_order_subtotal = sum(c.values())
    for sub_t, sub_c, in c.items():
         first_order[t][sub_t] = sub_c / first_order_subtotal
    
for t, c in second_order.items():    # new in ver2
    second_order_subtotal = sum(c.values())
    for sub_t, sub_c, in c.items():
         second_order[t][sub_t] = sub_c / second_order_subtotal
            
            
    
#second_order_total = sum(second_order.values())  # new in ver2
#for k, ts in second_order.items():
#    second_order[k] = c / second_order_total    

In [14]:
def sample_word(d): # sample a word given a dict of probabilities
    p0 = np.random.random()  # randomly chooses number (just once) between 0 and 1
    #print(f"p0 = {p0}")
    cumulative = 0
    for t, p in d.items():     # token and corresponding probability
        cumulative += p    # adds probability of each token
        #print(cumulative) 
        if cumulative > p0:    # originally if p0 < cumulative
            #print(t)
            return t
    assert(False) # should never get here

In [15]:
def generate():
    for i in range(4): # generate 4 lines at a time
        sentence = []
        
        # sample initital word
        w0 = sample_word(initial)
        sentence.append(w0)
        
        # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)
        
        # second-order transitions until END
        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == 'END':
                break   # goes to next line
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))  

In [16]:
generate()

if in fact it seems to be lifted
everything will come true in cyberspace things are simpler than they are male ego dominated
escape into the visual and the heart of the fractal laws that govern the world is something new and intelligent things to each other the insides of our lives and our collective past
the central figure in the offing the world at an informational level there is not culturally given to you is your responsibility because the only choice is real and what the goddess really is a prophetic dimension a glimpse of the potential of a neurotic culture against the dehumanizing values that are normally hidden from view
